<a href="https://colab.research.google.com/github/AnnaBsoares/AnnaBsoares/blob/main/cpppppIaaaa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.cluster import KMeans
from sklearn.metrics import mean_squared_error

Importamos bibliotecas essenciais para manipulação de dados (pandas), criação de modelos (RandomForestRegressor), escalonamento de dados (StandardScaler), imputação de valores ausentes (SimpleImputer), clusterização (KMeans) e avaliação do modelo (mean_squared_error).

# Tratar dados

In [2]:
# Função para tratar dados (imputação)
def trata_dados(dados):
    imputer = SimpleImputer(strategy='mean')
    dados_imputados = imputer.fit_transform(dados)
    return pd.DataFrame(dados_imputados, columns=dados.columns), imputer


A função trata_dados recebe um DataFrame e substitui valores ausentes pela média das respectivas colunas, retornando o DataFrame imputado e o objeto imputer usado para essa transformação.

In [3]:
# Carregar e preparar dados
dados = pd.read_csv('/content/solicitacoescredito.csv')
dados_sem_nan = dados.dropna(subset=['limiteEmpresaAnaliseCredito']).copy()


Os dados são carregados de um arquivo CSV.
As linhas que possuem valores ausentes na coluna limiteEmpresaAnaliseCredito são removidas.

# Selecionar características e alvo

In [4]:
# Selecionar características e alvo
caracteristicas = dados_sem_nan[['maiorAtraso', 'margemBrutaAcumulada', 'percentualProtestos', 'faturamentoBruto']]
alvo = dados_sem_nan['limiteEmpresaAnaliseCredito']


Selecionamos as características (variáveis independentes) e o alvo (variável dependente) do DataFrame.

# Imputar valores ausentes e padronizar dados

In [5]:
# Imputar valores ausentes e padronizar dados
caracteristicas, imputer = trata_dados(caracteristicas)
scaler = StandardScaler()
caracteristicas_padronizadas = scaler.fit_transform(caracteristicas)


Imputamos valores ausentes nas características usando a função trata_dados.
Padronizamos os dados (z-score normalization) para que todas as características tenham média 0 e desvio padrão 1.

# classificação usando KMeans
## Segmentação dos clientes usando KMeans

In [6]:
# Segmentação dos clientes usando KMeans
n_clusters = 3  # Defina o número de clusters conforme necessário
kmeans = KMeans(n_clusters=n_clusters, n_init=10, random_state=42)
rotulos_clusters = kmeans.fit_predict(caracteristicas_padronizadas)


Definimos o número de clusters (grupos).
Aplicamos KMeans para segmentar os clientes com base nas características padronizadas.

In [7]:
# Adicionar os rótulos dos clusters ao DataFrame usando .loc para evitar SettingWithCopyWarning
dados_sem_nan.loc[:, 'grupo_cliente'] = rotulos_clusters


Adicionamos os rótulos dos clusters ao DataFrame original para identificar a qual grupo cada cliente pertence.



In [8]:
# Exibir o DataFrame resultante
print(dados_sem_nan)


      numero_solicitacao             razaoSocial        nomeFantasia  \
0                      1  James Richardson-Patel  Alexandra Williams   
2                      3           Joanna Hudson      Dr. David Rees   
3                      4    Gordon Jones-Hopkins    Sara Reid-Robson   
4                      5               Nigel Lee  Dr. Stanley Duncan   
5                      6            Liam Jackson       Gerard Gordon   
...                  ...                     ...                 ...   
8958                9031              Joel Allan        Gavin Holden   
8959                9032          Douglas Taylor           Dale Lowe   
8960                9033               Jamie Cox        Kirsty Jones   
8961                9034             Sara Foster          Dean Price   
8963                9036               June Lane  Miss Sharon Peters   

            cnpjSemTraco  maiorAtraso  margemBrutaAcumulada  \
0     KEBE17609492220843            0              0.252448   
2     GCV

In [9]:
import numpy as np

# Função para calcular a raiz do erro quadrático médio (RMSE)
def root_mean_squared_error(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))


## Treinar um modelo para cada grupo

In [10]:
# Treinar um modelo para cada grupo
modelos_por_grupo = {}
for grupo in range(n_clusters):
    dados_grupo = dados_sem_nan[dados_sem_nan['grupo_cliente'] == grupo]
    X_grupo = dados_grupo[['maiorAtraso', 'margemBrutaAcumulada', 'percentualProtestos', 'faturamentoBruto']]
    y_grupo = dados_grupo['limiteEmpresaAnaliseCredito']

    X_grupo_imputado, _ = trata_dados(X_grupo)

    # Ajustar um novo StandardScaler para este grupo
    scaler_grupo = StandardScaler()
    X_grupo_padronizado = scaler_grupo.fit_transform(X_grupo_imputado)

    X_treino, X_val, y_treino, y_val = train_test_split(X_grupo_padronizado, y_grupo, test_size=0.2, random_state=42)

    modelo_grupo = RandomForestRegressor(random_state=42)
    modelo_grupo.fit(X_treino, y_treino)

    previsoes = modelo_grupo.predict(X_val)
    rmse = mean_squared_error(y_val, previsoes, squared=False)
    print(f'RMSE para o Grupo {grupo}: {rmse}')

    modelos_por_grupo[grupo] = modelo_grupo

RMSE para o Grupo 0: 7383094.580982074
RMSE para o Grupo 1: 40319.399999999994
RMSE para o Grupo 2: 56356141.322433755


Criamos dicionários para armazenar os escalonadores e modelos por grupo.
Para cada grupo, segmentamos os dados, imputamos valores ausentes e padronizamos os dados novamente com um escalonador específico para o grupo.
Treinamos um modelo RandomForestRegressor para cada grupo e calculamos o RMSE (erro quadrático médio) para avaliar a performance do modelo.

## Função para recomendar limite de crédito para um novo cliente

In [11]:
def recomendar_limite_credito_classificacao(novo_cliente, imputer, kmeans, modelos_por_grupo):
    # Criar DataFrame com o novo cliente
    novo_cliente_df = pd.DataFrame([novo_cliente])

    # Imputar valores ausentes no novo cliente
    novo_cliente_imputado = imputer.transform(novo_cliente_df)

    # Determinar o grupo do novo cliente usando KMeans
    novo_cliente_grupo = kmeans.predict(novo_cliente_imputado)[0]

    # Prever o limite de crédito usando o modelo do grupo correspondente
    limite_credito = modelos_por_grupo[novo_cliente_grupo].predict(novo_cliente_imputado)

    return limite_credito[0]

A função recomendar_limite_credito cria um DataFrame com os dados do novo cliente.
Imputa valores ausentes e padroniza os dados do novo cliente usando o escalonador geral.
Determina o grupo do novo cliente utilizando KMeans.
Padroniza os dados novamente usando o escalonador específico do grupo.
Usa o modelo do grupo correspondente para prever o limite de crédito.

## Defina um novo cliente classificaçao

In [12]:
# Recomendar limite de crédito usando a abordagem de classificação
limite_credito_classificacao = recomendar_limite_credito_classificacao(novo_cliente, imputer, kmeans, modelos_por_grupo)
print(f'Limite de crédito recomendado para o novo cliente (Classificação): {limite_credito_classificacao}')

NameError: name 'novo_cliente' is not defined

Definimos um novo cliente com suas características.
Chamamos a função recomendar_limite_credito para prever o limite de crédito do novo cliente.
Imprimimos o limite de crédito recomendado.

## Acurácia

In [ ]:
from sklearn.metrics import accuracy_score

# Dividir os dados em conjunto de treino e teste
X_treino, X_teste, y_treino, y_teste = train_test_split(caracteristicas_padronizadas, rotulos_clusters, test_size=0.2, random_state=42)

# Treinar o modelo de regressão (RandomForestRegressor)
modelo_regressao = RandomForestRegressor(random_state=42)
modelo_regressao.fit(X_treino, y_treino)

# Fazer previsões no conjunto de teste para a classificação
previsoes_teste_classificacao = modelo_regressao.predict(X_teste)

# Calcular a acurácia
acuracia = accuracy_score(y_teste, previsoes_teste_classificacao)
print(f'Acurácia no conjunto de teste: {acuracia}')

# Modelo de Regressão

In [ ]:
# Dividir os dados em conjunto de treino e teste
X_treino, X_teste, y_treino, y_teste = train_test_split(caracteristicas_padronizadas, alvo, test_size=0.2, random_state=42)

# Treinar o modelo de regressão (RandomForestRegressor)
modelo_regressao = RandomForestRegressor(random_state=42)
modelo_regressao.fit(X_treino, y_treino)

# Avaliar o modelo usando RMSE no conjunto de teste
previsoes_teste = modelo_regressao.predict(X_teste)
rmse_teste = mean_squared_error(y_teste, previsoes_teste, squared=False)
print(f'RMSE no conjunto de teste (Regressão): {rmse_teste}')

# Função para recomendar o limite de crédito para um novo cliente usando regressão
def recomendar_limite_credito_regressao(novo_cliente):
    # Imputar valores ausentes no novo cliente
    novo_cliente_imputado = imputer.transform(pd.DataFrame([novo_cliente]))

    # Determinar o limite de crédito usando o modelo de regressão
    limite_credito = modelo_regressao.predict(novo_cliente_imputado)

    return limite_credito[0]

# Avaliar o modelo usando RMSE no conjunto de teste
previsoes_teste = modelo_regressao.predict(X_teste)
rmse_teste = mean_squared_error(y_teste, previsoes_teste, squared=False)
print(f'RMSE no conjunto de teste (Regressão): {rmse_teste}')

# Função para recomendar o limite de crédito para um novo cliente usando regressão
def recomendar_limite_credito_regressao(novo_cliente):
    # Imputar valores ausentes no novo cliente
    novo_cliente_imputado = imputer.transform(pd.DataFrame([novo_cliente]))

    # Determinar o limite de crédito usando o modelo de regressão
    limite_credito = modelo_regressao.predict(novo_cliente_imputado)

    return limite_credito[0]

# Definir um novo cliente para teste
novo_cliente = [30, 0.2, 5, 10000]

In [ ]:
# Recomendar limite de crédito usando a abordagem de regressão
limite_credito_regressao = recomendar_limite_credito_regressao(novo_cliente)
print(f'Limite de crédito recomendado para o novo cliente (Regressão): {limite_credito_regressao}')

## performance do modelo de regressão.

In [ ]:

# Avaliar o modelo usando RMSE no conjunto de teste
previsoes_teste = modelo_regressao.predict(X_teste)
rmse_teste = mean_squared_error(y_teste, previsoes_teste, squared=False)
print(f'RMSE no conjunto de teste: {rmse_teste}')

#  dashboard para o modelo de classificação usando KMeans:

> Este é o código para o dashboard do modelo de classificação usando KMeans. Ele permite que você insira as características de um novo cliente e, ao clicar no botão "Calcular", mostrará o limite de crédito recomendado com base no modelo de classificação usando KMeans.


In [ ]:
!pip install dash


In [ ]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd

# Carregar os dados
dados = pd.read_csv('/content/solicitacoescredito.csv')

# Definir o aplicativo Dash
app = dash.Dash(__name__)

# Layout do aplicativo para o modelo de classificação usando KMeans
app.layout = html.Div([
    html.H1('Dashboard - Modelo de Classificação (KMeans)'),
    html.Div([
        html.Label('Maior Atraso'),
        dcc.Input(id='input1', type='number', placeholder='Maior Atraso'),
        html.Label('Margem Bruta Acumulada'),
        dcc.Input(id='input2', type='number', placeholder='Margem Bruta Acumulada'),
        html.Label('Percentual Protestos'),
        dcc.Input(id='input3', type='number', placeholder='Percentual Protestos'),
        html.Label('Faturamento Bruto'),
        dcc.Input(id='input4', type='number', placeholder='Faturamento Bruto'),
        html.Button('Calcular', id='calcular-button'),
        html.Div(id='limite-credito-output')
    ])
])

# Callback para calcular o limite de crédito recomendado usando KMeans
@app.callback(
    Output('limite-credito-output', 'children'),
    [Input('calcular-button', 'n_clicks')],
    [dash.dependencies.State('input1', 'value'),
     dash.dependencies.State('input2', 'value'),
     dash.dependencies.State('input3', 'value'),
     dash.dependencies.State('input4', 'value')]
)
def calcular_limite_credito_kmeans(n_clicks, maior_atraso, margem_bruta_acumulada, percentual_protestos, faturamento_bruto):
    if n_clicks is not None:
        # Aqui você colocaria a lógica para calcular o limite de crédito usando KMeans
        limite_credito_recomendado = 10000  # Apenas um valor de exemplo para demonstração
        return f'Limite de Crédito Recomendado (KMeans): {limite_credito_recomendado}'

    return ''

# Executar o aplicativo
if __name__ == '__main__':
    app.run_server(debug=True)


# Dashboard para o modelo de regressão usando RandomForestRegressor

In [ ]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd

# Carregar os dados
dados = pd.read_csv('/content/solicitacoescredito.csv')

# Definir o aplicativo Dash
app = dash.Dash(__name__)

# Layout do aplicativo para o modelo de regressão usando RandomForestRegressor
app.layout = html.Div([
    html.H1('Dashboard - Modelo de Regressão (Random Forest)'),
    html.Div([
        html.Label('Maior Atraso'),
        dcc.Input(id='input1', type='number', placeholder='Maior Atraso'),
        html.Label('Margem Bruta Acumulada'),
        dcc.Input(id='input2', type='number', placeholder='Margem Bruta Acumulada'),
        html.Label('Percentual Protestos'),
        dcc.Input(id='input3', type='number', placeholder='Percentual Protestos'),
        html.Label('Faturamento Bruto'),
        dcc.Input(id='input4', type='number', placeholder='Faturamento Bruto'),
        html.Button('Calcular', id='calcular-button'),
        html.Div(id='limite-credito-output')
    ])
])

# Função para recomendar limite de crédito para um novo cliente
def recomendar_limite_credito(novo_cliente):
    # Padronizar os dados do novo cliente
    novo_cliente_imputado = imputer.transform(pd.DataFrame([novo_cliente]))
    novo_cliente_padronizado = scaler.transform(novo_cliente_imputado)

    # Fazer previsões usando o modelo de regressão
    limite_credito_recomendado = modelo_regressao.predict(novo_cliente_padronizado)

    return limite_credito_recomendado[0]

# Callback para calcular o limite de crédito recomendado usando RandomForestRegressor
@app.callback(
    Output('limite-credito-output', 'children'),
    [Input('calcular-button', 'n_clicks')],
    [dash.dependencies.State('input1', 'value'),
     dash.dependencies.State('input2', 'value'),
     dash.dependencies.State('input3', 'value'),
     dash.dependencies.State('input4', 'value')]
)
def calcular_limite_credito_regressao(n_clicks, maior_atraso, margem_bruta_acumulada, percentual_protestos, faturamento_bruto):
    if n_clicks is not None:
        novo_cliente = {
            'maiorAtraso': maior_atraso,
            'margemBrutaAcumulada': margem_bruta_acumulada,
            'percentualProtestos': percentual_protestos,
            'faturamentoBruto': faturamento_bruto
        }
        limite_credito_recomendado = recomendar_limite_credito(novo_cliente)
        return f'Limite de Crédito Recomendado (Regressão): {limite_credito_recomendado}'

    return ''

# Executar o aplicativo
if __name__ == '__main__':
    app.run_server(debug=True)


# Obtendo a importância das características para cada grupo

* Este código irá calcular e imprimir as importâncias das características para cada grupo separadamente. Cada valor de importância representa a contribuição relativa de uma característica na previsão do limite de crédito para o grupo correspondente.



In [ ]:
# Criar um dicionário para armazenar as importâncias das características para cada grupo
importancias_por_grupo = {}

# Calcular a importância das características para cada modelo de grupo
for grupo, modelo_grupo in modelos_por_grupo.items():
    importancias_caracteristicas = modelo_grupo.feature_importances_
    importancias_por_grupo[grupo] = importancias_caracteristicas

# Imprimir as importâncias das características para cada grupo
for grupo, importancias in importancias_por_grupo.items():
    print(f'Grupo {grupo}:')
    for i, importancia in enumerate(importancias):
        print(f'    Característica {i+1}: {importancia}')


In [ ]:
# Convertendo para DataFrame para visualização
df_importancias_caracteristicas = pd.DataFrame(importancias_por_grupo, index=X_grupo.columns)

# Plotando as importâncias das características para cada grupo
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(10, 6))
df_importancias_caracteristicas.plot(kind='bar', ax=ax)
plt.title('Importância das Características por Grupo')
plt.xlabel('Características')
plt.ylabel('Importância')
plt.xticks(rotation=45)
plt.legend(title='Grupo')
plt.tight_layout()
plt.show()


# Vizualização
## 1. Matriz de Correlação
   
* No contexto do conjunto de dados de solicitações de crédito, a matriz de correlação nos permite entender como as diferentes características (como atrasos no pagamento, margem bruta acumulada, percentual de protestos e faturamento bruto) estão relacionadas entre si.
* Por exemplo, se houver uma alta correlação positiva entre o percentual de protestos e o atraso no pagamento, isso pode indicar que clientes com mais protestos tendem a ter mais atrasos nos pagamentos.



In [ ]:
import seaborn as sns

# Calcular a matriz de correlação
correlation_matrix = dados_sem_nan[['maiorAtraso', 'margemBrutaAcumulada', 'percentualProtestos', 'faturamentoBruto', 'limiteEmpresaAnaliseCredito']].corr()

# Plotar a matriz de correlação
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", annot_kws={"size": 12})
plt.title('Matriz de Correlação')
plt.show()



# Scatter Plots

* Os scatter plots nos ajudam a visualizar a relação entre duas características específicas para diferentes grupos de clientes.
* Por exemplo, podemos usar um scatter plot para ver como o percentual de protestos se relaciona com o atraso no pagamento, e se essa relação varia entre os diferentes grupos de clientes identificados pelo algoritmo de clusterização.


In [ ]:

import seaborn as sns
import matplotlib.pyplot as plt


# Scatter Plots
sns.pairplot(dados_sem_nan, hue='grupo_cliente', vars=['maiorAtraso', 'margemBrutaAcumulada', 'percentualProtestos', 'faturamentoBruto'])
plt.title('Scatter Plots')
plt.show()




# Gráficos de Violino:

* Os gráficos de violino nos permitem visualizar a distribuição das características em cada grupo de clientes, mostrando tanto a tendência central quanto a dispersão dos dados.
* Por exemplo, podemos usar gráficos de violino para entender como a margem bruta acumulada varia entre os diferentes grupos de clientes e identificar se há diferenças significativas em suas distribuições.

In [ ]:
# Gráficos de Violino
plt.figure(figsize=(10, 6))
sns.violinplot(x='grupo_cliente', y='maiorAtraso', data=dados_sem_nan)
plt.title('Gráfico de Violino para "maiorAtraso" por Grupo')
plt.show()


#Gráficos de Barra:

* Os gráficos de barra nos ajudam a comparar a média de uma característica específica entre os diferentes grupos de clientes.
* Por exemplo, podemos usar um gráfico de barra para comparar a média da margem bruta acumulada entre os grupos de clientes e identificar se há diferenças significativas em suas médias.

In [ ]:
# Gráficos de Barra
plt.figure(figsize=(10, 6))
sns.barplot(x='grupo_cliente', y='margemBrutaAcumulada', data=dados_sem_nan, estimator=np.mean)
plt.title('Média de "margemBrutaAcumulada" por Grupo')
plt.show()
